In [1]:
import os
from transformers import BertTokenizer, AutoTokenizer
from transexp_orig.ExplanationGenerator import Generator
from transexp_orig.BertForSequenceClassification import BertForSequenceClassification
import torch

/root/anaconda3/envs/cutoff/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2022-11-19 08:08:23.480379: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [3]:
import transformers
transformers.__version__

'4.24.0'

In [2]:
data_name = "WNLI"

In [ ]:
model = BertForSequenceClassification.from_pretrained(f"textattack/roberta-base-{data_name}").to("cuda")
model.eval()
tokenizer = AutoTokenizer.from_pretrained(f"textattack/roberta-base-{data_name}")
tokenizer_baseline = AutoTokenizer.from_pretrained(f"roberta-base")
# tokenizer = AutoTokenizer.from_pretrained(f"textattack/bert-base-uncased-{data_name}")
# initialize the explanations generator
explanations = Generator(model)

classifications = ["NEGATIVE", "POSITIVE"]


In [4]:
from transformers_cutoff.data.processors.glue import glue_convert_examples_to_features, glue_output_modes, glue_processors
processor = glue_processors[data_name.lower()]()
examples = (processor.get_train_examples(f"/home/jovyan/work/datasets/{data_name}"))

In [ ]:
batch_encoding = tokenizer.batch_encode_plus(
        [(example.text_a, example.text_b) for example in examples], max_length=128, pad_to_max_length=True, #truncation=True
    )
batch_encoding_baseline = tokenizer_baseline.batch_encode_plus(
        [(example.text_a, example.text_b) for example in examples], max_length=128, pad_to_max_length=True, #truncation=True
    )

In [23]:
from transformers_cutoff.data.processors.utils import InputFeatures
from typing import List, Optional, Union
from transformers_cutoff.data.processors.utils import DataProcessor, InputExample

label_list = processor.get_labels()
label_map = {label: i for i, label in enumerate(label_list)}
labels = [label_map[example.label] for example in examples]

features = []
for i in range(len(examples)):
  inputs = {k: batch_encoding[k][i] for k in batch_encoding}

  feature = InputFeatures(**inputs, label=labels[i])
  features.append(feature)

In [24]:
features_baseline = []
for i in range(len(examples)):
  inputs_bl = {k: batch_encoding_baseline[k][i] for k in batch_encoding_baseline}

  feature_bl = InputFeatures(**inputs_bl, label=labels[i])
  features_baseline.append(feature_bl)

In [35]:
tokenizer.decode(features[0].input_ids)
# print(" ".join(converted))

'<s>I stuck a pin through a carrot. When I pulled the pin out, it had a hole.</s></s>The carrot had a hole.</s><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad>'

In [40]:
tokenizer_baseline.decode(features[15].input_ids)

'<s>I poured water from the bottle into the cup until it was empty.</s></s>The cup was empty.</s><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad>'

In [7]:
from transformers_cutoff.data.processors.glue import glue_convert_examples_to_features, glue_output_modes, glue_processors
from transformers_cutoff.data.processors.utils import InputFeatures, InputAugFeatures
from transformers_cutoff.tokenization_roberta import RobertaTokenizer, RobertaTokenizerFast
from transformers_cutoff.tokenization_utils import PreTrainedTokenizer
from transformers_cutoff.tokenization_xlm_roberta import XLMRobertaTokenizer

In [8]:
import logging
from typing import List, Optional, Union
from transformers_cutoff.data.processors.utils import DataProcessor, InputExample, InputFeatures
from transformers_cutoff.file_utils import is_tf_available

if is_tf_available():
    import tensorflow as tf
    
logger = logging.getLogger(__name__)
    
def _glue_convert_examples_to_features(
    examples: List[InputExample],
    tokenizer: PreTrainedTokenizer,
    max_length: Optional[int] = None,
    task=None,
    label_list=None,
    output_mode=None,
):
    """
        generate tokens as features
    """
    if max_length is None:
        max_length = tokenizer.max_len
    
    if task is not None:
        processor = glue_processors[task]()
        if label_list is None:
            label_list = processor.get_labels()
            logger.info("Using label list %s for task %s" % (label_list, task))
        if output_mode is None:
            output_mode = glue_output_modes[task]
            logger.info("Using output mode %s for task %s" % (output_mode, task))

    label_map = {label: i for i, label in enumerate(label_list)}

    def label_from_example(example: InputExample) -> Union[int, float]:
        if output_mode == "classification":
            return label_map[example.label]
        elif output_mode == "regression":
            return float(example.label)
        raise KeyError(output_mode)

    labels = [label_from_example(example) for example in examples]

    batch_encoding = tokenizer.batch_encode_plus(
        [(example.text_a, example.text_b) for example in examples], max_length=max_length, pad_to_max_length=True,
    )

    features = []
    for i in range(len(examples)):
        inputs = {k: batch_encoding[k][i] for k in batch_encoding}

        feature = InputFeatures(**inputs, label=labels[i])
        features.append(feature)

    for i, example in enumerate(examples[:5]):
        logger.info("*** Example ***")
        logger.info("guid: %s" % (example.guid))
        logger.info("features: %s" % features[i])

    return features

def glue_convert_examples_to_features(
    examples: Union[List[InputExample], "tf.data.Dataset"],
    tokenizer: PreTrainedTokenizer,
    max_length: Optional[int] = None,
    task=None,
    label_list=None,
    output_mode=None,
):
    """
    Loads a data file into a list of ``InputFeatures``

    Args:
        examples: List of ``InputExamples`` or ``tf.data.Dataset`` containing the examples.
        tokenizer: Instance of a tokenizer that will tokenize the examples
        max_length: Maximum example length. Defaults to the tokenizer's max_len
        task: GLUE task
        label_list: List of labels. Can be obtained from the processor using the ``processor.get_labels()`` method
        output_mode: String indicating the output mode. Either ``regression`` or ``classification``

    Returns:
        If the ``examples`` input is a ``tf.data.Dataset``, will return a ``tf.data.Dataset``
        containing the task-specific features. If the input is a list of ``InputExamples``, will return
        a list of task-specific ``InputFeatures`` which can be fed to the model.

    """
    if is_tf_available() and isinstance(examples, tf.data.Dataset):
        if task is None:
            raise ValueError("When calling glue_convert_examples_to_features from TF, the task parameter is required.")
        return _tf_glue_convert_examples_to_features(examples, tokenizer, max_length=max_length, task=task)
    return _glue_convert_examples_to_features(
        examples, tokenizer, max_length=max_length, task=task, label_list=label_list, output_mode=output_mode
    )


if is_tf_available():

    def _tf_glue_convert_examples_to_features(
        examples: tf.data.Dataset, tokenizer: PreTrainedTokenizer, task=str, max_length: Optional[int] = None,
    ) -> tf.data.Dataset:
        """
        Returns:
            A ``tf.data.Dataset`` containing the task-specific features.

        """
        processor = glue_processors[task]()
        examples = [processor.tfds_map(processor.get_example_from_tensor_dict(example)) for example in examples]
        features = glue_convert_examples_to_features(examples, tokenizer, max_length=max_length, task=task)

        def gen():
            for ex in features:
                yield (
                    {
                        "input_ids": ex.input_ids,
                        "attention_mask": ex.attention_mask,
                        "token_type_ids": ex.token_type_ids,
                    },
                    ex.label,
                )

        return tf.data.Dataset.from_generator(
            gen,
            ({"input_ids": tf.int32, "attention_mask": tf.int32, "token_type_ids": tf.int32}, tf.int64),
            (
                {
                    "input_ids": tf.TensorShape([None]),
                    "attention_mask": tf.TensorShape([None]),
                    "token_type_ids": tf.TensorShape([None]),
                },
                tf.TensorShape([]),
            ),
        )

In [10]:
processor = glue_processors[data_name.lower()]()
output_mode = glue_output_modes[data_name.lower()]
label_list = processor.get_labels()

if data_name in ["mnli", "mnli-mm"] and tokenizer.__class__ in (
      RobertaTokenizer,
      RobertaTokenizerFast,
      XLMRobertaTokenizer,
  ):
  label_list[1], label_list[2] = label_list[2], label_list[1]
  
examples = (
    processor.get_train_examples(f"/home/jovyan/work/datasets/{data_name}")
)
  
features = glue_convert_examples_to_features(
                    examples,
                    tokenizer,
                    max_length=128,
                    label_list=label_list,
                    output_mode=output_mode,
                )
  

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/root/anaconda3/envs/cutoff/lib/python3.8/site-packages/transformers/tokenization_utils_base.py:2304: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


In [11]:
features

[InputFeatures(input_ids=[101, 1045, 5881, 1037, 9231, 2083, 1037, 25659, 1012, 2043, 1045, 2766, 1996, 9231, 2041, 1010, 2009, 2018, 1037, 4920, 1012, 102, 1996, 25659, 2018, 1037, 4920, 1012, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], attention_mask=[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], token_type_ids=[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1,

In [4]:
# encode a sentence
text_batch = ["This movie was the best movie I have ever seen! some scenes were ridiculous, but acting was great."]
encoding = tokenizer(text_batch, return_tensors='pt')
input_ids = encoding['input_ids'].to("cuda")
attention_mask = encoding['attention_mask'].to("cuda")

# true class is positive - 1
true_class = 1

# generate an explanation for the input
expl = explanations.generate_LRP(input_ids=input_ids, attention_mask=attention_mask, start_layer=0)[0]
# normalize scores
expl = (expl - expl.min()) / (expl.max() - expl.min())

# get the model classification
output = torch.nn.functional.softmax(model(input_ids=input_ids, attention_mask=attention_mask)[0], dim=-1)
classification = output.argmax(dim=-1).item()
# get class name
class_name = classifications[classification]
# if the classification is negative, higher explanation scores are more negative
# flip for visualization
if class_name == "NEGATIVE":
  expl *= (-1)

tokens = tokenizer.convert_ids_to_tokens(input_ids.flatten())
print([(tokens[i], expl[i].item()) for i in range(len(tokens))])
vis_data_records = [visualization.VisualizationDataRecord(
                                expl,
                                output[0][classification],
                                classification,
                                true_class,
                                true_class,
                                1,       
                                tokens,
                                1)]
visualization.visualize_text(vis_data_records)

/root/anaconda3/envs/cutoff/lib/python3.8/site-packages/transformers/modeling_utils.py:763: FutureWarning: The `device` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(


[('[CLS]', 0.0), ('this', 0.4254930019378662), ('movie', 0.30647191405296326), ('was', 0.26705053448677063), ('the', 0.31505000591278076), ('best', 0.6269277930259705), ('movie', 0.28255173563957214), ('i', 0.1865611970424652), ('have', 0.10077141225337982), ('ever', 0.14222446084022522), ('seen', 0.18850412964820862), ('!', 0.5975315570831299), ('some', 0.0038841008208692074), ('scenes', 0.033900078386068344), ('were', 0.017799649387598038), ('ridiculous', 0.020366042852401733), (',', 0.0), ('but', 0.42744413018226624), ('acting', 0.4414099156856537), ('was', 0.4980330765247345), ('great', 1.0), ('.', 0.016455795615911484), ('[SEP]', 0.08527109771966934)]


True Label,Predicted Label,Attribution Label,Attribution Score,Word Importance
1,1 (1.00),1,1.00,"[CLS] this movie was the best movie i have ever seen ! some scenes were ridiculous , but acting was great . [SEP]"


True Label,Predicted Label,Attribution Label,Attribution Score,Word Importance
1,1 (1.00),1,1.00,"[CLS] this movie was the best movie i have ever seen ! some scenes were ridiculous , but acting was great . [SEP]"
